In [1]:
import os 
import numpy as np
import string
import torch
import re
from tqdm import tqdm
from scipy.io import loadmat
from utils import  TrainDataset

## Dataset
The dataset contains the EEG recordings of 15 subjects. For each subject, we have 15 different recordings, each one collected while watching a different movie clip. Each clip is associated to an emotional state amon {sad: -1, neutral: 0, happy: 1}. EEG recordings comprises 62 channels.

N.B. Recordings correspondent to the same movies have the same length, while recordings correspondent to different movies have different length (iun general). How to do? No problem, since we are taking sub windows of the signals.

Data have been preprocessed by downsampling signals to 200Hz, segmentating the signals such that it corresponds to the length of the movie and applying a band-pass filter at 0-75Hz. Since recordings are about 4 minutes long and are now sampled at 200Hz, they contain roughly 48k time points each.

In [3]:
data = loadmat("data/Preprocessed_EEG/1_20131027.mat")

In [4]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'djc_eeg1', 'djc_eeg2', 'djc_eeg3', 'djc_eeg4', 'djc_eeg5', 'djc_eeg6', 'djc_eeg7', 'djc_eeg8', 'djc_eeg9', 'djc_eeg10', 'djc_eeg11', 'djc_eeg12', 'djc_eeg13', 'djc_eeg14', 'djc_eeg15'])

In [ ]:
labels = loadmat("data/Preprocessed_EEG/label.mat")
labels["label"].shape

In [ ]:
print(data.keys())
data["djc_eeg3"].shape

In [ ]:
eeg_files = [fname for fname in os.listdir("data/Preprocessed_EEG") if fname[0] in string.digits]
data_lengths = []
for eeg_file in tqdm(eeg_files):
    raw = loadmat(os.path.join("data/Preprocessed_EEG", eeg_file))
    curr_lengths = []
    pattern = list(raw.keys())[4].split("_")[0]
    for i in range(15):
        curr_lengths.append(raw[f"{pattern}_eeg{i + 1}"].shape[1])
    data_lengths.append(curr_lengths)

data_lengths = np.asarray(data_lengths)
print(data_lengths.shape)
print(np.mean(data_lengths))
print(np.mean(data_lengths, axis=0))
print(np.mean(data_lengths, axis=1))
print(np.unique(data_lengths, return_counts=True))
print(np.std(data_lengths, axis=0))

In [2]:
dataset = TrainDataset("data/Preprocessed_EEG", "data/Preprocessed_EEG/label.mat", 2000, 200, False)

Loading data files: 100%|██████████| 45/45 [01:06<00:00,  1.47s/it]


In [22]:
import torch
import torch.nn as nn

# Define a simple input tensor with 1 channel and size 4x4
torch.random.manual_seed(1234)
input_tensor = torch.randn(1, 2, 4, 4)  # (batch_size, channels, height, width)

# Create a Conv2d layer with 2 groups
conv_layer = nn.Conv2d(in_channels=2, out_channels=4, kernel_size=3, groups=2)
print(conv_layer.weight)

# Perform the convolution
output_tensor = conv_layer(input_tensor)

# Print the shapes of the input and output tensors
print("Input tensor shape:", input_tensor.shape)
print("Output tensor shape:", output_tensor.shape)
input_tensor, output_tensor


Parameter containing:
tensor([[[[ 0.1929, -0.1189,  0.0165],
          [ 0.1126,  0.2291, -0.0490],
          [ 0.3041, -0.2820, -0.0595]]],


        [[[-0.3324,  0.0276,  0.0946],
          [-0.1349,  0.1384, -0.0540],
          [-0.2897,  0.2559,  0.2055]]],


        [[[ 0.1685,  0.2659,  0.1226],
          [ 0.1772,  0.2766, -0.0672],
          [-0.2600, -0.1639, -0.0445]]],


        [[[-0.0366, -0.0023,  0.1910],
          [ 0.1069, -0.2465, -0.1001],
          [-0.0784,  0.2029, -0.1209]]]], requires_grad=True)
Input tensor shape: torch.Size([1, 2, 4, 4])
Output tensor shape: torch.Size([1, 4, 2, 2])


(tensor([[[[-0.1117, -0.4966,  0.1631, -0.8817],
           [ 0.0539,  0.6684, -0.0597, -0.4675],
           [-0.2153,  0.8840, -0.7584, -0.3689],
           [-0.3424, -1.4020,  0.3206, -1.0219]],
 
          [[ 0.7988, -0.0923, -0.7049, -1.6024],
           [ 0.2891,  0.4899, -0.3853, -0.7120],
           [-0.1706, -1.4594,  0.2207,  0.2463],
           [-1.3248,  0.6970, -0.6631,  1.2158]]]]),
 tensor([[[[-0.2068,  0.3199],
           [ 0.2781, -0.5233]],
 
          [[ 0.2064, -0.5666],
           [-0.0600, -0.2478]],
 
          [[ 0.4253, -0.1238],
           [-0.1421, -0.5327]],
 
          [[-0.6064, -0.0351],
           [ 0.4775, -0.8064]]]], grad_fn=<ConvolutionBackward0>))

In [32]:
64 * 12 * 40 * 128 + 128 * 3 + 16 * 64 + 16 * 8 * 32 + 32 * 32 * 16 + 32 * 32

3955072